# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [27]:
import dask.array as da
import h5py
import numpy as np


1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [28]:
with h5py.File("./data/minutes_n_ingredients_full.hdf5", "r") as hdf:
	arr = np.asarray(hdf.get("recipe"))
	arr = da.asarray(arr, chunks=(100_000, 3))

In [29]:
arr

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [30]:
arr.compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      60,       2],
       [ 713836,       0,       9],
       [ 660699,      64,       8]])

In [31]:
arr[:, 1:].mean(axis=0).compute()

array([1004.20805176,    5.4198008 ])

In [32]:
# arr[:, 1].mean().compute()
# arr[:, 2].mean().compute()

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [33]:
from time import time
def get_chunk_time(chunk):
    with h5py.File("./data/minutes_n_ingredients_full.hdf5", 'r') as hdf:
        arr = np.asarray(hdf.get("recipe"))
        da_arr = da.from_array(arr, chunks=chunk)
    st = time()
    da_arr[:, 1:].mean(axis=0).compute()
    print(time() - st)

get_chunk_time((100, 3))
get_chunk_time((1000, 3))
# get_chunk_time((10000, 3))
# get_chunk_time((50000, 3))
# get_chunk_time((100000, 3))


9.111566066741943
1.0903279781341553


4. Выберите рецепты, время выполнения которых меньше медианного значения

In [34]:
median = da.median(arr[:, 1], axis=0).compute()
median

32.0

In [35]:
mask = arr[:, 1] < median
arr[mask].compute()
# arr[mask].compute().shape


array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]])

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [36]:
import dask.dataframe as dd

In [ ]:
df = dd.from_array(arr)
d = df.groupby(2)[0].count().compute().to_dict()
d

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [38]:
df[1].quantile(0.75).compute()

49.0

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [39]:
find_arr = da.array([9, 11])
arr[(da.power(arr[:, [1,2]] - find_arr, 2)).mean(axis=1).argmin()].compute()

array([316085,      9,     11])

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [58]:
def get_mean_but_block(filename):
    blocksize = 2000
    tmp = []
    with h5py.File(filename, 'r') as hdf:
        arr = hdf.get("recipe")
        
        for i in range(0, len(arr), blocksize):
            da_arr = da.from_array(arr[i:i+blocksize], chunks=(blocksize, 3))
            tmp.append(da_arr.mean(axis=0)[1])
    res_arr = da.stack(tmp, axis=0)
    
    return (res_arr.mean(axis=0)).compute()

In [59]:
get_mean_but_block("./data/minutes_n_ingredients_full.hdf5")

1004.0517107560272